In [2]:
# df_games = pd.read_csv("games.csv")
# df_teams = pd.read_csv("teams.csv")
# df_teamstats = pd.read_csv("teamstats.csv")
# df_leagues = pd.read_csv("leagues.csv")
# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/shots.csv")

df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["homeProbability", "drawProbability", "awayProbability"], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)

#df2_appearances = pd.read_csv("players/appearances.csv")
#df2_club_games = pd.read_csv("players/club_games.csv")
#df2_clubs = pd.read_csv("players/clubs.csv")
#df2_competitions = pd.read_csv("players/competitions.csv")
#df2_games = pd.read_csv("players/games.csv")
#df2_player_val= pd.read_csv("players/player_valuations.csv")
#df2_players = pd.read_csv("players/players.csv")
#df2_players.info()

df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv")


top_5_ligas=df2_competitions.loc[(df2_competitions['pretty_name']=='Bundesliga') | (df2_competitions['pretty_name']=='Laliga') | (df2_competitions['pretty_name']=='Premier League') | (df2_competitions['pretty_name']=='Serie A') | (df2_competitions['pretty_name']=='Ligue 1') ]
#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein
df_f_game[['date','hour']]=df_f_game['date'].str.split(' ',1,expand=True)
pd.options.mode.chained_assignment = None  # default='warn'

from fuzzywuzzy import fuzz, process


from thefuzz import process, fuzz

def match(x,df,score):
  match=process.extractOne(x, df,score_cutoff=score,scorer=fuzz.partial_ratio)
  if match:
    return match[2]
  else:
    return 2

top_5_ligas['new_id'] = df_teams.loc[top_5_ligas["pretty_name"].map(lambda x: match(x,df_teams.name,70) ).values,"name"].values
   # See note below

seasons=df_f_game["season"].unique()


df2_games= df2_games[df2_games['season'].isin(seasons)]
df2_games_top5= df2_games[df2_games['competition_id'].isin(top_5_ligas['competition_id'])]



df2_clubs_top5=df2_clubs[df2_clubs['domestic_competition_id'].isin(top_5_ligas['competition_id'])]

df2_clubs_top5['new_id'] = df_teams.loc[df2_clubs_top5['pretty_name'].map(lambda x: match(x,df_teams.name,80) ).values,"teamID"].values



df2_appearances_top5= df2_appearances[df2_appearances['game_id'].isin(df2_games_top5['game_id'])]


df2_players_top5=df2_players[df2_players['current_club_id'].isin(df2_clubs_top5['club_id'])]

#df2_players_top5['new_id'] = df_players.loc[df2_players_top5['pretty_name'].map(lambda x: match(x,df_players.name,80) ).values,"name"].values



df2_players_val_top5=df2_player_val[df2_player_val['player_id'].isin(df2_players_top5['player_id'])]



df_teams
df2_clubs_top5.to_excel('prueba.xlsx')
df2_players_top5.to_excel('prueba_players.xlsx')

df_teams
df2_clubs_top5.pretty_name
lista1=("dfasdf","sdf","#$3","fsadfd","asdfa","#sdf","asdfddd","asdfasdf")
lista2=("dfasdf","sdf","#$3","fsadfd","asdfa","#sdf","asdfddd","asdfasdf")

match = df_teams.loc[df2_clubs_top5["pretty_name"].map(lambda x: match(x,df_teams.name,95) ).values,'teamID'].values

c:\Users\Mati\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
c:\Users\Mati\AppData\Local\Programs\Python\Python310\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
